In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abstract/sample_submission.csv
/kaggle/input/abstract/train.csv
/kaggle/input/abstract/test.csv


In [ ]:
# !pip install -Uqq fastcore --upgrade
# !pip install -Uqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
# import fastcore

In [3]:
trainData = pd.read_csv('/kaggle/input/abstract/train.csv', sep=',', header="infer")
testData = pd.read_csv('/kaggle/input/abstract/test.csv', sep = ',', header = "infer")
testLabels = pd.read_csv('/kaggle/input/abstract/sample_submission.csv', sep = ',', header = "infer")

from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(trainData, train_size=0.9, random_state=34)
X_train, X_test, y_train, y_test  = train_test_split(train_data.ABSTRACT.values, train_data.iloc[:,3:].values,train_size=0.7, random_state=34)

In [46]:
train_data.iloc[:,3:].values

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0]])

# Implementation of ULMFIT In FastAI

In [4]:
# import fastai
# !pip install torch==1.8.0
# import torch
from fastai.text.all import *
# from fastai.core. import 
# import pandas as pd
# import numpy as np
# from functools import partial
# import io
# import os

In [ ]:
# dls = pd.DataFrame({'text': X_train, 'label': list(y_train)})
# dls_clas = TextDataLoaders.from_df(dls, valid='test', is_lm=True)
# classifier = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, Perplexity()], loss_func =FocalLossFlat(gamma=2))

In [5]:
# create a dataframe so as to pass it to TextDataLoaders with text and label as columns
dls_lm = pd.DataFrame({'text': train_data.iloc[:,2].values, 'label': list(train_data.iloc[:,3:].values)})

# This will automatically process all text and split and add special tokens like xxbos - to indicate beginning of text, xxmaj - indicate next word was capitalized 
data_lm = TextDataLoaders.from_df(dls_lm, label_delim=',' ,is_lm=True, valid_pct=0.1, bs=16)

#Have a glimpse of data
# data_lm.show_batch(max_n=5)

# ->Language Model trained for one epoch on last layer

In [6]:
# A pretrained language model is trying to learn from the above processed data
learn = language_model_learner(data_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.492076,4.222342,0.291437,68.193008,31:42


# ->Unfreezing all layers Language Model for fine tuning for 3 epochs

In [7]:
learn.save('1epoch')
learn = learn.load('1epoch')
learn.unfreeze()
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.098483,3.988292,0.326126,53.962654,30:47
1,4.081359,3.950693,0.330911,51.971390,31:30


# ->Save The model

In [8]:
learn.save_encoder('finetuned')

# We can use this Language Model for generating sentences as it is trained to guess what next word is.

In [ ]:
TEXT = "We investigate the formation and early evolution of star clusters assuming"
N_WORDS = 50
N_SENTENCES = 3
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))

# Training a text classifier involves training the last few layers.

In [32]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj some have praised xxunk xxmaj lost xxmaj xxunk as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think xxmaj the xxmaj dirty xxmaj dozen ) but xxunk represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before ,",neg


In [30]:
# dls_clas_test.show_batch(max_n=5)
# len(data_lm.vocab)
dls_clas.show_batch(max_n=3)

,text,text_
0,"xxbos xxmaj we present the extension of the effective field theory framework to the \n mildly non - linear scales . xxmaj the effective field theory approach has been \n successfully applied to the late time cosmic acceleration phenomenon and it has \n been shown to be a powerful method to obtain predictions about cosmological \n observables on linear scales . xxmaj however , mildly non - linear scales need to be","xxmaj we present the extension of the effective field theory framework to the \n mildly non - linear scales . xxmaj the effective field theory approach has been \n successfully applied to the late time cosmic acceleration phenomenon and it has \n been shown to be a powerful method to obtain predictions about cosmological \n observables on linear scales . xxmaj however , mildly non - linear scales need to be \n"
1,"two non - linear systems that are strictly outside the class \n considered in this paper . xxbos xxmaj graph based semi - supervised learning ( gssl ) has intuitive representation and \n can be improved by exploiting the matrix calculation . xxmaj however , it has to \n perform iterative optimization to achieve a preset objective , which usually \n leads to low efficiency . xxmaj another inconvenience lying in xxup","non - linear systems that are strictly outside the class \n considered in this paper . xxbos xxmaj graph based semi - supervised learning ( gssl ) has intuitive representation and \n can be improved by exploiting the matrix calculation . xxmaj however , it has to \n perform iterative optimization to achieve a preset objective , which usually \n leads to low efficiency . xxmaj another inconvenience lying in xxup gssl"
2,"to the body of the \n mouse . xxbos xxmaj we consider a dual - hop wireless network where an energy constrained relay \n node first harvests energy through the received radio - frequency signal from the \n source , and then uses the harvested energy to forward the source 's information \n to the destination node . xxmaj the throughput and delay metrics are investigated for \n a decode - and","the body of the \n mouse . xxbos xxmaj we consider a dual - hop wireless network where an energy constrained relay \n node first harvests energy through the received radio - frequency signal from the \n source , and then uses the harvested energy to forward the source 's information \n to the destination node . xxmaj the throughput and delay metrics are investigated for \n a decode - and -"


In [2]:
# # import numpy as np
# # # list(y_train[0])
# y_train_sol = []

# for i in y_train:
#     y_train_sol.append(list(i))
    
# print(np.squeeze(y_train_sol)[0])
# len(y_train_sol)
# # str(['what', 'is', 'this']).spilt(',')
train_data_cp = train_data.copy()
train_data_cp.iloc[:,3] = train_data.iloc[:,3].apply(lambda x: 'Computer Science' if x == 1 else '')
train_data_cp.iloc[:,4] = train_data.iloc[:,4].apply(lambda x: 'Physics' if x == 1 else '')
train_data_cp.iloc[:,5] = train_data.iloc[:,5].apply(lambda x: 'Mathematics' if x == 1 else '')
train_data_cp.iloc[:,6] = train_data.iloc[:,6].apply(lambda x: 'Statistics' if x == 1 else '')
train_data_cp.iloc[:,7] = train_data.iloc[:,7].apply(lambda x: 'Quantitative Biology' if x == 1 else '')
train_data_cp.iloc[:,8] = train_data.iloc[:,8].apply(lambda x: 'Quantitative Finance' if x == 1 else '')
list(train_data_cp.iloc[:,3:].values)
    

NameError: name 'train_data' is not defined

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer
MLB = MultiLabelBinarizer()
mlb.fit_transform([(1, 2), (3,)])

Index(['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
      dtype='object')

In [18]:
dls = pd.DataFrame({'text': X_train, 'category': np.squeeze(y_train_sol)})
# dls_clas = TextDataLoaders.from_csv(path='',text_col=0, valid_pct=0.2, label_col=1,label_delim=",",vocab=data_lm.train_ds.vocab, bs=32)

# learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, Perplexity()], loss_func =FocalLossFlat(gamma=2))

clsf = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=[accuracy, Perplexity()], loss_func =FocalLossFlat(gamma=2))

ValueError: Data must be 1-dimensional

In [39]:
len(y_train)

2

In [42]:
learn = learn.load_encoder('finetuned')
# learn.fit_one_cycle(1, 2e-2, cbs=[ShowGraphCallback()])
# learn.freeze_to(-2)
# learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), cbs=[ShowGraphCallback()])
# # learn.freeze_to(-3)
# # learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), cbs=[ShowGraphCallback()])
# learn.unfreeze()
# learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), cbs=[ShowGraphCallback()])

RuntimeError: Error(s) in loading state_dict for AWD_LSTM:
	size mismatch for encoder.weight: copying a param with shape torch.Size([27136, 400]) from checkpoint, the shape in current model is torch.Size([22496, 400]).
	size mismatch for encoder_dp.emb.weight: copying a param with shape torch.Size([27136, 400]) from checkpoint, the shape in current model is torch.Size([22496, 400]).